# 문제 6

[Kaggle 형] train_prob.csv로 문제 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
#pip install --upgrade mlxtend 
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [5]:
df_train = pd.read_csv('train_prob.csv', index_col=['id'])
df_test = pd.read_csv('test_prob.csv', index_col=['id'])
# 실제시험 x
s_ans = pd.read_csv('test_prob_ans.csv', index_col=['id'])['failure']

In [6]:
df_train = df_train.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna(),
)
df_test = df_test.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna(),
)

In [14]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.linear_model import LinearRegression
from sklearn.impute import IterativeImputer

# 방법 2: groupby ~ apply ~ fit_transform
# product_code 별로 IterativeImputer를 통한 결측처리를 하고 바로 적용합니다.
imp  = IterativeImputer(
    estimator=LinearRegression(fit_intercept=True), 
    random_state=123
)
X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']

# apply에서 transform에서 numpy array를 넘겨 주므로, 
# DataFrame을 만들어 반환시켜주면,
# apply에서 DataFrame으로 재구성하여 넘겨줍니다.
# 모델을 적용하여 결측이 처리된 데이터프레임을 만듭니다.
df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=X_imp)
)
df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=X_imp)
)

# 방법 3: groupby ~ transform
# Transform을 통해 수준별 평균으로 1:1 변환된 값으로 구성된 DataFrame을 받아서, 
# 각각의 요소별로 결측이면 해당값으로 치환되게 구성합니다. 
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] =  df_train.groupby('product_code')[X_mean].transform(
    lambda x: x.fillna(x.mean())
)
df_test[X_mean] =  df_test.groupby('product_code')[X_mean].transform(
    lambda x: x.fillna(x.mean())
)

In [16]:
df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
df_test['loading'] = df_test['loading'].fillna(df_train['loading'].mean())

In [7]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [17]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64



- loading의 각 행들에 자연 로그 함수를 적용하여 파생 변수 loading_log를 만든다.

- LR + SFS: \['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1'\] 최적 변수

- LDA: transform / predict

- LR + PCA: n_components=7 + 'loading'

- RF: {'n_estimators': 15, 'max_depth': 7, 'min_samples_split': 512}, ['loading'] + ['measurement_{}'.format(i) for i in range(18)] + ['na_1', 'na_2']



# Kaggle형 풀이 단계

Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Step 2: Baseline 모델을 만듭니다

Step 3: 모델 선택 루틴을 만듭니다.

Step 4: 모델 개선 작업을 합니다.

In [22]:
from sklearn.model_selection import GroupKFold, cross_validate

gcv = GroupKFold(n_splits=4)
for train_idx, valid_idx in gcv.split(df_train, df_train['failure'], groups=df_train['product_code']):
    print(
        df_train.iloc[train_idx]['product_code'].unique(), 
        df_train.iloc[valid_idx]['product_code'].unique()
    )

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

cross_validate(
    make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs')),
    df_train[['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1']],
    df_train['failure'],
    cv = gcv,
    groups=df_train['product_code'], 
    scoring='roc_auc',
    return_train_score=True
)

{'fit_time': array([0.0404253 , 0.02318096, 0.03029585, 0.04003239]),
 'score_time': array([0.00387049, 0.01046252, 0.        , 0.        ]),
 'test_score': array([0.58823117, 0.58496642, 0.58887652, 0.59535198]),
 'train_score': array([0.59259755, 0.5935028 , 0.59192673, 0.58955959])}